In [123]:
import pandas as pd 
import random 
from sklearn.utils import shuffle

In [245]:
df = pd.read_csv('../data/geotargets-2020-03-03.csv')

## UK data df_1

In [3]:
uk_abbr = pd.read_excel('../data/abbr.xlsx', sheet_name = 'uk')
uk_abbr_map = dict(zip(uk_abbr['Country'], uk_abbr['Abbreviation']))

In [4]:
def abbr_map(x, map_dict): #input a cell with state
    x[1] = map_dict[x[1]]
    return x

In [5]:
df_1 = df[(df['Country Code']=='GB') & (df['Target Type'] == 'City') & (df['Status'] == 'Active')]
df_1 = df_1.reset_index(drop = True)

df_1['name_split_0'] = df_1['Canonical Name'].apply(lambda x: x.split(','))
df_1['name_split_0'] = df_1['name_split_0'].apply(lambda x: x[1:] if len(x)>3 else x)
df_1['name_split_1'] = df_1['name_split_0'].apply(lambda x: [x[0], x[-1]])
df_1['name_split_2'] = df_1['name_split_1'].apply(lambda x: x[:-1]+['UK'])
df_1['name_split_3'] = df_1['name_split_0'].apply(lambda x: x[:-1]+['UK'])
df_1['name_split_4'] = df_1['name_split_1'].apply(lambda x: x[:-1]+['GB'])
df_1['name_split_5'] = df_1['name_split_0'].apply(lambda x: x[:-1]+['GB'])
df_1['name_split_6'] = df_1['name_split_0'].apply(lambda x: [x[0], 'Great Britain'])
df_1['name_split_7'] = df_1['name_split_0'].apply(lambda x: x[:-1])
df_1['name_split_8'] = df_1['name_split_0'].apply(lambda x: abbr_map(x, uk_abbr_map))
df_1['name_split_9'] = df_1['name_split_3'].apply(lambda x: abbr_map(x, uk_abbr_map))
df_1['name_split_10'] = df_1['name_split_5'].apply(lambda x: abbr_map(x, uk_abbr_map))

In [461]:
def get_combinations(split_type_length, dataframe): 
    col_select = list(range(0,split_type_length+1))
    str_1_list = []
    str_2_list = []
    for idx, row in dataframe.iterrows(): 
        split_type = random.sample(col_select, 2)
        if type(row['name_split_'+str(split_type[0])]) == list: 
                str_1_list.append(', '.join(row['name_split_'+str(split_type[0])]))
                str_2_list.append(', '.join(row['name_split_'+str(split_type[1])]))
        else: 
                str_1_list.append(row['name_split_'+str(split_type[0])])
                str_2_list.append(row['name_split_'+str(split_type[1])])                  
    return str_1_list, str_2_list

In [463]:
all_df_1 = pd.DataFrame(columns = ['str_1', 'str_2'])
uk = get_combinations(10, df_1)
all_df_1['str_1'] = uk[0]
all_df_1['str_2'] = uk[1]

In [464]:
all_df_2 = pd.DataFrame(columns = ['str_1', 'str_2'])
uk = get_combinations(10, df_1)
all_df_2['str_1'] = uk[0]
all_df_2['str_2'] = uk[1]

In [465]:
all_df_1 = pd.concat([all_df_1, all_df_2])

In [621]:
all_df_1 = shuffle(all_df_1)
all_df_1 = all_df_1.head(1000)
all_df_1.shape

(1000, 2)

In [622]:
def generate_neg(current_df, num_neg, str_1_length): 
    current_df = current_df.reset_index(drop = True)
    neg_df_return = pd.DataFrame(columns = ['str_1', 'str_2'])
    neg_sample = random.sample(list(current_df.index), str_1_length)
    for idx in neg_sample: 
        temp_neg_df = pd.DataFrame(columns = ['str_1', 'str_2'])
        temp_str_1 = current_df.drop([idx])['str_1']
        temp_str_1 = shuffle(temp_str_1).head(str_1_length)
        temp_str_2 = current_df.iat[idx, 1]
        temp_neg_df['str_1'] = temp_str_1
        temp_neg_df['str_2'] = temp_str_2
        neg_df_return = pd.concat([neg_df_return, temp_neg_df])
    neg_df_return['Y'] = 0
    return neg_df_return

In [623]:
neg_df_1 = generate_neg(all_df_1, 10, 20)

## US data df_2

In [246]:
df_2 = df[(df['Country Code']=='US') & (df['Target Type'] == 'City') & (df['Status'] == 'Active')]
df_2 = df_2.reset_index(drop = True)

In [247]:
us_abbr = pd.read_excel('../data/abbr.xlsx', sheet_name = 'us')
us_abbr_map = dict(zip(us_abbr['state'], us_abbr['abbr']))

In [250]:
df_2['name_split_0'] = df_2['Canonical Name'].apply(lambda x: x.split(','))
df_2['name_split_0'] = df_2['name_split_0'].apply(lambda x: x[1:] if len(x)>3 else x)
df_2['name_split_1'] = df_2['Canonical Name'].apply(lambda x: x.split(','))
df_2['name_split_1'] = df_2['name_split_1'].apply(lambda x: x[1:] if len(x)>3 else x)
df_2['name_split_1'] = df_2['name_split_1'].apply(lambda x: x[:-1]+['US'])
df_2['name_split_2'] = df_2['name_split_1'].apply(lambda x: x[:-1]+['USA'])
df_2['name_split_3'] = df_2.apply(lambda x: x['name_split_2'][:-1], axis = 1)
df_2['name_split_4'] = df_2['name_split_2'].apply(lambda x: abbr_map(x, us_abbr_map))

In [252]:
df_2['name_split_4'] = df_2.apply(lambda x: x['name_split_4'][:-1], axis = 1)

In [624]:
all_df_2 = pd.DataFrame(columns = ['str_1', 'str_2'])
us = get_combinations(4, df_2)
all_df_2['str_1'] = us[0]
all_df_2['str_2'] = us[1]

In [625]:
all_df_2 = shuffle(all_df_2).head(1000)
all_df_2.shape

(1000, 2)

In [626]:
neg_df_2 = generate_neg(all_df_2, 5, 20)

## Company name df_3

In [361]:
MOCK_1 = pd.read_csv('../data/MOCK_DATA_1.csv')

In [591]:
comp_df = MOCK_1[['company_name']].reset_index(drop = True)

In [592]:
comp_df['name_split_0'] = comp_df['company_name'].apply(lambda x: x.upper())
comp_df['ending'] = comp_df['name_split_0'].apply(lambda x: x.split()[-1])
comp_df['start'] = comp_df['name_split_0'].apply(lambda x: x.split()[:-1])

In [593]:
comp_dict = {'CORPORATION': 'Corp.', 'TRUST': 'Tr.', 'DIVIDEND': 'DIV', 'INC.': 'Incorporated', \
             'BANK': 'BK', 'CORP.': 'Corporation','Incorporated': 'Inc.', 'BANCORP': 'Corporation', \
            'LTD.': 'Limited', 'L.P.': 'Limited Partnership', 'BANCORP.': 'Corporation', \
            'LIMITED': 'Ltd.', 'GROUP': 'Association', 'LTD': 'LIMITED', 'CO.': 'Company'}

In [594]:
comp_df['name_split_1'] = comp_df['name_split_0'].apply(lambda x: x.replace('&', ' and '))

In [595]:
comp_df['replace'] = comp_df['ending'].map(comp_dict)
comp_df['replace'] = comp_df['replace'].fillna('')

In [596]:
comp_df['name_split_2'] = comp_df.apply(lambda row: row['start']+[row['replace']], axis = 1)
comp_df['name_split_2'] = comp_df['name_split_2'].apply(lambda x: ' '.join(x))
comp_df['name_split_3'] = comp_df['name_split_2'].apply(lambda x: x.replace('&', ' and '))

In [600]:
all_df_3 = pd.DataFrame(columns = ['str_1', 'str_2'])
company = get_combinations(3, comp_df)
all_df_3['str_1'] = company[0]
all_df_3['str_2'] = company[1]

In [601]:
all_df_3.shape

(1000, 2)

In [615]:
neg_df_3 = generate_neg(all_df_3, 20, 30)

In [616]:
neg_df_3

,str_1,str_2,Y
35,"TELEPHONE AND DATA SYSTEMS, INC.","CATERPILLAR, INC.",0
492,WESTERN ASSET/CLAYMORE U.S TREASURY INFLATION ...,"CATERPILLAR, INC.",0
110,"ACLARIS THERAPEUTICS, Incorporated","CATERPILLAR, INC.",0
9,JENSYN ACQUISTION CORP.,"CATERPILLAR, INC.",0
225,WESTERN ASSET EMERGING MARKETS DEBT FUND INC,"CATERPILLAR, INC.",0
...,...,...,...
765,"TELEPHONE AND DATA SYSTEMS, INC.",DLH HOLDINGS Corporation,0
826,"WASHINGTON TRUST BANCORP, Incorporated",DLH HOLDINGS Corporation,0
558,"PROTO LABS, INC.",DLH HOLDINGS Corporation,0
707,"AUTODESK, Incorporated",DLH HOLDINGS Corporation,0


## US address df_4

In [387]:
df_4 = MOCK_1[['address', 'state', 'state_abbr']].reset_index(drop=True)

In [388]:
# get a city/ town name for each location
locations = df_2['Canonical Name'].tolist()
locations = [x.split(',')[0] for x in locations]
locations = random.sample(locations, 1000)
df_4['city'] = locations

In [389]:
# get postal code for each location

In [390]:
df_4['street_type'] = df_4['address'].apply(lambda x: x.split()[-1])

In [391]:
street_dict = {'Point':'Pt', 'Drive':'Dr', 'Junction':'Jct', 'Court':'Ct', 'Lane':'Ln', 'Street':'St', 'Avenue':'Av',
       'Way':'Way', 'Trail':'Trl', 'Terrace':'Ter', 'Center':'Ctr', 'Crossing':'XING', 'Park':'Park', 'Road':'Rd',
       'Circle':'Cir', 'Pass':'Pass', 'Parkway':'Pkwy', 'Hill':'Hl', 'Place':'Pl', 'Alley':'Aly', 'Plaza':'Plz'}
df_4['street_type_abbr'] = df_4['street_type'].map(street_dict)

In [392]:
def get_address_2(row): 
    address = row['address'].split()
    address[2] = row['street_type_abbr']
    return ' '.join(address)
df_4['address_2'] = df_4.apply(lambda row: get_address_2(row), axis = 1)

In [393]:
df_4['name_split_0'] = df_4['address']+', '+df_4['city']+', '+df_4['state']
df_4['name_split_1'] = df_4['address_2']+', '+df_4['city']+', '+df_4['state']
df_4['name_split_2'] = df_4['address']+', '+df_4['city']+', '+df_4['state_abbr']
df_4['name_split_3'] = df_4['address_2']+', '+df_4['city']+', '+df_4['state_abbr']

In [394]:
code = list(range(0, 10))*5
postal = []
for i in range(0, 1000): 
    p = [str(x) for x in random.sample(code, 5)]
    postal.append(''.join(p))
df_4['postal_code'] = postal

In [395]:
df_4['name_split_4'] = df_4['name_split_1']+' '+df_4['postal_code']
df_4['name_split_5'] = df_4['name_split_2']+' '+df_4['postal_code']
df_4['name_split_6'] = df_4['name_split_3']+' '+df_4['postal_code']
df_4['name_split_7'] = df_4['name_split_4']+', USA'
df_4['name_split_8'] = df_4['name_split_5']+', USA'

In [396]:
all_df_4 = pd.DataFrame()
all_df_4_2 = pd.DataFrame()

In [397]:
gen_1 = get_combinations(8, df_4)
all_df_4['str_1'] = gen_1[0]
all_df_4['str_2'] = gen_1[1]

gen_2 = get_combinations(8, df_4)
all_df_4_2['str_1'] = gen_1[0]
all_df_4_2['str_2'] = gen_1[1]

In [398]:
all_df_4 = pd.concat([all_df_4, all_df_4_2])

In [421]:
all_df_4.shape

(2000, 2)

In [516]:
neg_df_4 = generate_neg(all_df_4, 5, 20)

In [517]:
neg_df_4

,str_1,str_2,Y
1882,"6379 Cottonwood Pt, Baxter, LA","071 Anniversary Crossing, Ross, Florida",0
1839,"86 Vermont Park, Elgin, LA 19187","071 Anniversary Crossing, Ross, Florida",0
964,"3946 Dexter Terrace, Spring Arbor, LA 58680, USA","071 Anniversary Crossing, Ross, Florida",0
812,"3 Hauk Hl, Blackwater, TX","071 Anniversary Crossing, Ross, Florida",0
1142,"9513 Grover Terrace, Mitchell, NC 58255, USA","071 Anniversary Crossing, Ross, Florida",0
...,...,...,...
1402,"69195 Coolidge Center, Lucerne, KS","8400 5th Rd, Elbow Lake, LA",0
1785,"384 Logan Pkwy, Lecanto, IL","8400 5th Rd, Elbow Lake, LA",0
520,"8968 Luster Dr, Abbeville, Florida 78221, USA","8400 5th Rd, Elbow Lake, LA",0
553,"11 Melby Trl, Floyds Knobs, IL","8400 5th Rd, Elbow Lake, LA",0


## UK address df_5

In [374]:
MOCK_2 = pd.read_csv('../data/MOCK_DATA_2.csv')
df_5 = MOCK_2

In [376]:
# get a city/ town name for each locatio
locations = df_1['Canonical Name'].tolist()
locations = [x.split(',')[0] for x in locations]
locations_1 = random.sample(locations, 900)
locations_2 = random.sample(locations, 100)
locations = locations_1+locations_2
df_5['city'] = locations

In [381]:
df_5['street_type'] = df_5['address'].apply(lambda x: x.split()[-1])
df_5['street_type_abbr'] = df_5['street_type'].map(street_dict)
df_5['address_2'] = df_5.apply(lambda row: get_address_2(row), axis = 1)
df_5['name_split_0'] = df_5['address']+', '+df_5['city']+', '+df_5['state']
df_5['name_split_1'] = df_5['address_2']+', '+df_5['city']+', '+df_5['state']
df_5['name_split_2'] = df_5['address']+', '+df_5['city']+', '+df_5['state_abbr']
df_5['name_split_3'] = df_5['address_2']+', '+df_5['city']+', '+df_5['state_abbr']
df_5['name_split_4'] = df_5['name_split_1']+', '+df_5['postal_code']
df_5['name_split_5'] = df_5['name_split_2']+', '+df_5['postal_code']
df_5['name_split_6'] = df_5['name_split_3']+', '+df_5['postal_code']
df_5['name_split_7'] = df_5['name_split_4']+', GB'
df_5['name_split_8'] = df_5['name_split_5']+', UK'

In [383]:
all_df_5 = pd.DataFrame()
all_df_5_2 = pd.DataFrame()

In [385]:
gen_1 = get_combinations(8, df_5)
all_df_5['str_1'] = gen_1[0]
all_df_5['str_2'] = gen_1[1]

gen_2 = get_combinations(8, df_5)
all_df_5_2['str_1'] = gen_1[0]
all_df_5_2['str_2'] = gen_1[1]

all_df_5 = pd.concat([all_df_5, all_df_5_2])

In [424]:
all_df_5.shape

(2000, 2)

In [518]:
neg_df_5 = generate_neg(all_df_5, 5, 20)

In [519]:
neg_df_5

,str_1,str_2,Y
1669,"3972 Monument Terrace, Porthcawl, ENG, RH5, UK","03 Kingsford Drive, Borehamwood, SCT, IV1, UK",0
1641,"3 Carberry Aly, Windsor, England, WC1B","03 Kingsford Drive, Borehamwood, SCT, IV1, UK",0
55,"332 Daystar Hill, Bredbury, SCT, AB39, UK","03 Kingsford Drive, Borehamwood, SCT, IV1, UK",0
1719,"074 Kim Plz, Roslin, NIR","03 Kingsford Drive, Borehamwood, SCT, IV1, UK",0
779,"44 Crowley Drive, Hemel Hempstead, ENG","03 Kingsford Drive, Borehamwood, SCT, IV1, UK",0
...,...,...,...
1825,"30 Acker Park, Willenhall, England, CB4, GB","92023 Debs Pl, Hexham, England",0
251,"5645 Dakota Lane, Watford, ENG, M34","92023 Debs Pl, Hexham, England",0
636,"52 Superior Way, Alconbury, ENG, NE46","92023 Debs Pl, Hexham, England",0
692,"52 Brown Park, Oswestry, SCT, KW10","92023 Debs Pl, Hexham, England",0


## Generate negative samples, concat and shuffle

In [627]:
all_df = pd.concat([all_df_1, all_df_2, all_df_3, all_df_4, all_df_5])
all_df['Y'] = 1
all_df = all_df[all_df['str_1']!=all_df['str_2']].reset_index(drop = True)

In [628]:
all_df

,str_1,str_2,Y
0,"Aberdeen, SCT, UK","Aberdeen, SCT, United Kingdom",1
1,"Kidderminster, GB","Kidderminster, England",1
2,"Keele, ENG, GB","Keele, GB",1
3,"Cowes, ENG, GB","Cowes, Great Britain",1
4,"Gloucester, Great Britain","Gloucester, United Kingdom",1
...,...,...,...
6574,"2559 Westridge Rd, Bourton, England","2559 Westridge Rd, Bourton, ENG",1
6575,"5 Vidon Alley, Bury, SCT, EH9, UK","5 Vidon Alley, Bury, SCT, EH9",1
6576,"782 Luster Av, Hailsham, England, LS6, GB","782 Luster Avenue, Hailsham, England",1
6577,"30 Jackson Rd, Tuxford, England, NG22, GB","30 Jackson Road, Tuxford, ENG, NG22, UK",1


In [629]:
neg_df = generate_neg(all_df, 30, 100)

df_train = pd.concat([all_df, neg_df])
df_train = shuffle(df_train)

In [630]:
neg_df

,str_1,str_2,Y
4839,"71 Magdeline Dr, Runcorn, ENG","3 Jenifer Pkwy, Converse, Florida 11398, USA",0
503,"Northwich, United Kingdom","3 Jenifer Pkwy, Converse, Florida 11398, USA",0
1758,"Climax Springs, Missouri, United States","3 Jenifer Pkwy, Converse, Florida 11398, USA",0
2624,"5869 Buhler Dr, Rutland, Texas","3 Jenifer Pkwy, Converse, Florida 11398, USA",0
5498,"91028 5th Rd, Rustington, England","3 Jenifer Pkwy, Converse, Florida 11398, USA",0
...,...,...,...
6032,"91173 Carioca XING, Tonbridge, ENG, BS41","271 Bluejay Center, Coatbridge, England",0
3932,"4283 Brown Plz, Chesnee, Virginia 26271","271 Bluejay Center, Coatbridge, England",0
733,"Oldbury, ENG, GB","271 Bluejay Center, Coatbridge, England",0
5483,"6 Sauthoff Lane, Wombwell, ENG","271 Bluejay Center, Coatbridge, England",0


In [631]:
other_neg_df = pd.concat([neg_df_1, neg_df_2, neg_df_3, neg_df_4, neg_df_5])

In [632]:
df_train = pd.concat([df_train, other_neg_df])
df_train = shuffle(df_train)

In [633]:
df_train['Y'].sum()/df_train.shape[0]

0.3448293935740867

In [620]:
df_train.to_csv('../data/traning_v5.csv')